# Melhorando strategy com decorator

In [2]:
from collections import namedtuple


Cliente = namedtuple("Cliente", "nome fidelidade")


class ProdutoNoCarrinho:
    def __init__(self, nome, quantidade, preco) -> None:
        self.nome = nome
        self.quantidade = quantidade
        self.preco = preco

    def total(self):
        return self.preco * self.quantidade


class Pedido:
    def __init__(
        self, cliente: Cliente, carrinho: list[ProdutoNoCarrinho], promocao=None
    ) -> None:
        self.cliente = cliente
        self.carrinho = carrinho
        self.promocao = promocao
        self.desconto = 0.0

    def total(self):
        if not hasattr(self, "__total"):
            self.__total = sum(item.total() for item in self.carrinho)
        return self.__total

    def desconte(self):
        if self.promocao is not None:
            self.desconto = self.promocao.desconto(self)

    def total_considerando_descontos(self):
        if self.promocao is None:
            return self.total()
        self.desconte()
        return self.total() - self.desconto

    def __repr__(self) -> str:
        self.desconte()
        linhas = [
            ("Subtotal", f"  R$ {self.total():7.2f}"),
            (" Desconto", f"- R$ {self.desconto:7.2f}"),
            (" Subtotal", f"  R$ {self.total_considerando_descontos():7.2f}"),
        ]
        return "<" + "\n".join(f"{tipo}: {valor}" for tipo, valor in linhas) + ">"

In [3]:
promos = []

#### Decorator #####
def promo(promo_func):
    """O único objetivo deste decorator é inserir a função na lista"""
    promos.append(promo_func)
    return promo_func
#####################

####### Funções #####
@promo
def fidelidade(pedido: Pedido) -> float:
    return pedido.total() * 0.05 if pedido.cliente.fidelidade >= 1000 else 0

@promo
def combinados(pedido: Pedido) -> float:
    return sum(p.total() for p in pedido.carrinho if p.quantidade >= 20) / 10

@promo
def grande(pedido: Pedido) -> float:
    return pedido.total() * .07 if len({p.nome for p in pedido.carrinho}) > 9 else 0

def melhor_promo(pedido: Pedido) -> float:
    return max(promo(pedido) for promo in promos)
#####################